# News Mood

In [2]:
# Dependencies
import tweepy
import json
import time
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns 
import numpy             as np
from config import consumer_key, consumer_secret, access_token, access_token_secret

In [3]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [4]:
# Twitter API Keys
consumer_key        = consumer_key
consumer_secret     = consumer_secret
access_token        = access_token
access_token_secret = access_token_secret

In [5]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api  = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [6]:
columns = ['Media', 'Tweet Index', 'TimeStamp', 'Compound', 'Positive', 'Neutral', 'Negative']

media_analysis_df = pd.DataFrame(columns=columns)
media_analysis_df.head()

,Media,Tweet Index,TimeStamp,Compound,Positive,Neutral,Negative


## API Call to Twitter

In [7]:
def parse_tweets(media, public_tweets):
    count = media_analysis_df.shape[0]
    
    # Loop through all tweets
    for index, tweet in enumerate(public_tweets):
        media_analysis_df.set_value(count, 'Media',       media)
        media_analysis_df.set_value(count, 'Tweet Index', index)
        media_analysis_df.set_value(count, 'TimeStamp',   tweet['created_at'])
        
        # Run Vader Analysis on each tweet
        compound = analyzer.polarity_scores(tweet["text"])["compound"]
        pos      = analyzer.polarity_scores(tweet["text"])["pos"]
        neu      = analyzer.polarity_scores(tweet["text"])["neu"]
        neg      = analyzer.polarity_scores(tweet["text"])["neg"]

        # Add each value to the appropriate array
        media_analysis_df.set_value(count, 'Compound', compound)
        media_analysis_df.set_value(count, 'Positive', pos)
        media_analysis_df.set_value(count, 'Neutral' , neu)
        media_analysis_df.set_value(count, 'Negative', neg)
        
        count += 1

In [8]:
# Target Search Term
tweet_limit   = 100
media_handles = {'BBC':            '@BBC',
                 'CBS':            '@CBS',
                 'CNN':            '@CNN',
                 'Fox':            '@FoxNews',
                 'New York Times': '@nytimes'}

for media, handle in media_handles.items():
    
    public_tweets = api.search(handle, count=tweet_limit, result_type='recent')
    parse_tweets(media, public_tweets['statuses'])

In [9]:
# media_analysis_df.to_csv('media_analysis.csv', index=False)
date = time.strftime("%m/%d/%Y")

if True:
    media_analysis_df = pd.read_csv('media_analysis.csv')
    date              = '04/08/2018'
    
media_analysis_df['Media'].value_counts()

Fox               100
BBC               100
CNN               100
New York Times    100
CBS               100
Name: Media, dtype: int64